In [1]:
%cd UQ-NLG/
import sys; sys.path.append("..")
from importlib import reload
import persist_to_disk as ptd
import os
ptd.config.set_project_path(os.path.abspath("../"))
import tqdm
import pandas as pd
import numpy as np
import torch.nn as nn

import re
import torch

import utils

import matplotlib.pyplot as plt
%matplotlib inline

CONTRADICT, NEUTRAL, AGREE = 0, 1, 2
# logits: [Contradiction, neutral, entailment]
# data_path = 'notebook/demo_data.pkl'
data_path = 'output_generations/gpt-3.5-turbo-0301_coqa_10/0.pkl'
demo_data = pd.read_pickle(data_path)

/data/user-data/sa25729/GPT/UQ-NLG


In [7]:
data_path_ = 'notebook/demo_data.pkl'
demo_data_ = pd.read_pickle(data_path_)
type(demo_data_[5]),demo_data_[5]

(dict,
 {'prompt': 'Answer these questions:\nQ: In Scotland a bothy/bothie is a?\nA: House\nQ: Director of Home Alone and the first two Harry Potter films and a 15th/16th century Genoese explorer.\nA:',
  'id': 'wh_119',
  'question': 'Director of Home Alone and the first two Harry Potter films and a 15th/16th century Genoese explorer.',
  'answer': 'CHRISTOPHER COLUMBUS',
  'generations': {'text_cleaned': [' John Hughes - Christopher Columbus',
    ' Chris Columbus and Cristoforo Colombo (Cristoforo Colombo, Columbus, Cristóbal Colón, etc, not to be confused with Cristopher Columbus)',
    ' Chris Columbus',
    ' Chris Columbus',
    ' Chris Columbus',
    ' Sir Chris Columbus and Christopher Columbus',
    ' Chris Columbus and John Cabot',
    ' Chris Columbus',
    ' John Hughes & Christopher Columbo',
    ' Chris Columbus or Christopher Columbus',
    ' Chris Columbus and Christopher Columbus',
    ' Chris Columbus/ Christopher Columbo',
    ' Chris Columbus',
    ' Chris Columbus

In [3]:
len(demo_data), demo_data[0]

(7983,
 {'prompt': 'Once upon a time, in a barn near a farm house, there lived a little white kitten named Cotton. Cotton lived high up in a nice warm place above the barn where all of the farmer\'s horses slept. But Cotton wasn\'t alone in her little home above the barn, oh no. She shared her hay bed with her mommy and 5 other sisters. All of her sisters were cute and fluffy, like Cotton. But she was the only white one in the bunch. The rest of her sisters were all orange with beautiful white tiger stripes like Cotton\'s mommy. Being different made Cotton quite sad. She often wished she looked like the rest of her family. So one day, when Cotton found a can of the old farmer\'s orange paint, she used it to paint herself like them. When her mommy and sisters found her they started laughing. \n\n"What are you doing, Cotton?!" \n\n"I only wanted to be more like you". \n\nCotton\'s mommy rubbed her face on Cotton\'s and said "Oh Cotton, but your fur is so pretty and special, like you. We 

In [4]:
data_path_ = 'output_generations/llama-13b-hf_coqa_10/0.pkl'
demo_data_ = pd.read_pickle(data_path_)
len(demo_data_), demo_data_[0]

(7983,
 {'prompt': tensor([    1,  9038,  2501,   263,   931, 29892,   297,   263,  2594, 29876,
           2978,   263, 17888,  3699, 29892,   727, 10600,   263,  2217,  4796,
            413, 16097,  4257, 26783,   880, 29889, 26783,   880, 10600,  1880,
            701,   297,   263,  7575, 14294,  2058,  2038,   278,  2594, 29876,
            988,   599,   310,   278,  2215,  1050, 29915, 29879, 15100, 12844,
            415, 29889,  1205, 26783,   880,  9007, 29915, 29873,  7432,   297,
            902,  2217,  3271,  2038,   278,  2594, 29876, 29892,  9360,   694,
          29889,  2296,  7258,   902, 14842,  6592,   411,   902, 16823,  1357,
            322, 29871, 29945,   916,  9883, 29879, 29889,  2178,   310,   902,
           9883, 29879,   892,   274,  1082,   322,  1652,  3096, 29891, 29892,
            763, 26783,   880, 29889,  1205,  1183,   471,   278,   871,  4796,
            697,   297,   278, 14928, 29889,   450,  1791,   310,   902,  9883,
          29879,   892,

In [5]:
data_path_0 = 'output_generations/opt-13b_coqa_10/0.pkl'
demo_data_0 = pd.read_pickle(data_path_0)
len(demo_data_0), demo_data_0[0]

(7983,
 {'prompt': tensor([    2, 11475,  2115,    10,    86,     6,    11,    10, 15909,   583,
             10,  3380,   790,     6,    89,  3033,    10,   410,  1104, 35292,
           1440, 15416,     4, 15416,  3033,   239,    62,    11,    10,  2579,
           3279,   317,  1065,     5, 15909,   147,    70,     9,     5, 10305,
             18,  8087, 17931,     4,   125, 15416,   938,    75,  1937,    11,
             69,   410,   184,  1065,     5, 15909,     6, 14223,   117,     4,
            264,  1373,    69, 13460,  3267,    19,    69,  3795,  4783,     8,
            195,    97,  7502,     4,   404,     9,    69,  7502,    58, 11962,
              8, 33564,     6,   101, 15416,     4,   125,    79,    21,     5,
            129,  1104,    65,    11,     5,  6900,     4,    20,  1079,     9,
             69,  7502,    58,    70,  8978,    19,  2721,  1104, 23921, 26224,
            101, 15416,    18,  3795,  4783,     4,  8374,   430,   156, 15416,
           1341,  5074,

In [20]:
def _clean_sample(sample, tokenizer):
    # https://github.com/lorenzkuhn/semantic_uncertainty/blob/main/code/clean_generated_strings.py
    def _clean_answer(old_text:str, old_token_ids, tokenizer):
        cleaned_text = old_text
        strings_to_filter_on = [
                    '.', '\n', 'Q:', 'A:', 'question:', 'answer:', 'Question:', 'Answer:', 'Questions:', 'questions:', 'QUESTION:',
                    'ANSWER:'
                ]
        for string in strings_to_filter_on:
            if string in cleaned_text:
                cleaned_text = cleaned_text.split(string)[0]
        if tokenizer is None:
            return dict(
                text_cleaned=cleaned_text,
                text=old_text,
            )
        token_ids = tokenizer.encode(cleaned_text, return_tensors='pt')[0]
        assert token_ids[0] == tokenizer.bos_token_id
        token_ids = token_ids[1:]
        return dict(text_cleaned=cleaned_text,
                    token_cleaned=token_ids.cpu(),
                    text=old_text,
                    token=old_token_ids.cpu(),
                    )
    ret = {k: sample[k] for k in ['prompt', 'id', 'question', 'answer', 'additional_answers']}
    ret['generations'] = [None] * len(sample['generations'])
    if tokenizer is None:
        for i, generation in enumerate(sample['generations']):
            ret['generations'][i] = _clean_answer(generation, None, tokenizer)
    else:
        for i, generation in enumerate(sample['generations_ids']):
            generation = generation[generation.ne(tokenizer.pad_token_id)]
            generation = generation[generation.ne(tokenizer.eos_token_id)]
            ret['generations'][i] = _clean_answer(sample['generations'][i], generation, tokenizer)
    ret['generations'] = {k: [v[k] for v in ret['generations']] for k in ret['generations'][0].keys()}
    return ret

new_sample = _clean_sample(demo_data_0[0], tokenizer=None)
len(new_sample['generations']['text_cleaned']), new_sample

(20,
 {'prompt': tensor([    2, 11475,  2115,    10,    86,     6,    11,    10, 15909,   583,
             10,  3380,   790,     6,    89,  3033,    10,   410,  1104, 35292,
           1440, 15416,     4, 15416,  3033,   239,    62,    11,    10,  2579,
           3279,   317,  1065,     5, 15909,   147,    70,     9,     5, 10305,
             18,  8087, 17931,     4,   125, 15416,   938,    75,  1937,    11,
             69,   410,   184,  1065,     5, 15909,     6, 14223,   117,     4,
            264,  1373,    69, 13460,  3267,    19,    69,  3795,  4783,     8,
            195,    97,  7502,     4,   404,     9,    69,  7502,    58, 11962,
              8, 33564,     6,   101, 15416,     4,   125,    79,    21,     5,
            129,  1104,    65,    11,     5,  6900,     4,    20,  1079,     9,
             69,  7502,    58,    70,  8978,    19,  2721,  1104, 23921, 26224,
            101, 15416,    18,  3795,  4783,     4,  8374,   430,   156, 15416,
           1341,  5074,  

In [22]:
import pickle
with open('text_cleaned.pkl', 'wb') as fp:
    pickle.dump(new_sample, fp)

In [23]:
pd.read_pickle('text_cleaned.pkl')

{'prompt': tensor([    2, 11475,  2115,    10,    86,     6,    11,    10, 15909,   583,
            10,  3380,   790,     6,    89,  3033,    10,   410,  1104, 35292,
          1440, 15416,     4, 15416,  3033,   239,    62,    11,    10,  2579,
          3279,   317,  1065,     5, 15909,   147,    70,     9,     5, 10305,
            18,  8087, 17931,     4,   125, 15416,   938,    75,  1937,    11,
            69,   410,   184,  1065,     5, 15909,     6, 14223,   117,     4,
           264,  1373,    69, 13460,  3267,    19,    69,  3795,  4783,     8,
           195,    97,  7502,     4,   404,     9,    69,  7502,    58, 11962,
             8, 33564,     6,   101, 15416,     4,   125,    79,    21,     5,
           129,  1104,    65,    11,     5,  6900,     4,    20,  1079,     9,
            69,  7502,    58,    70,  8978,    19,  2721,  1104, 23921, 26224,
           101, 15416,    18,  3795,  4783,     4,  8374,   430,   156, 15416,
          1341,  5074,     4,   264,   747

In [24]:
cleaned_sequences = [_clean_sample(sample, tokenizer=None) for sample in tqdm.tqdm(demo_data_0[:5])]


100%|███████████████████████████████████████████| 5/5 [00:00<00:00, 8272.79it/s]


In [30]:
len(cleaned_sequences), type(cleaned_sequences), type(demo_data_0)

(5, list, list)

In [31]:
import pickle
with open('text_cleaned_.pkl', 'wb') as fp:
    pickle.dump(cleaned_sequences, fp)

In [32]:
temp = pd.read_pickle('text_cleaned_.pkl')
len(temp), temp[0]

(5,
 {'prompt': tensor([    2, 11475,  2115,    10,    86,     6,    11,    10, 15909,   583,
             10,  3380,   790,     6,    89,  3033,    10,   410,  1104, 35292,
           1440, 15416,     4, 15416,  3033,   239,    62,    11,    10,  2579,
           3279,   317,  1065,     5, 15909,   147,    70,     9,     5, 10305,
             18,  8087, 17931,     4,   125, 15416,   938,    75,  1937,    11,
             69,   410,   184,  1065,     5, 15909,     6, 14223,   117,     4,
            264,  1373,    69, 13460,  3267,    19,    69,  3795,  4783,     8,
            195,    97,  7502,     4,   404,     9,    69,  7502,    58, 11962,
              8, 33564,     6,   101, 15416,     4,   125,    79,    21,     5,
            129,  1104,    65,    11,     5,  6900,     4,    20,  1079,     9,
             69,  7502,    58,    70,  8978,    19,  2721,  1104, 23921, 26224,
            101, 15416,    18,  3795,  4783,     4,  8374,   430,   156, 15416,
           1341,  5074,   

In [34]:
data_path_0 = 'output_generations/opt-13b_coqa_10/0.pkl'
demo_data_0 = pd.read_pickle(data_path_0)
print(len(demo_data_0))

cleaned_sequences_opt_coqa = [_clean_sample(sample, tokenizer=None) for sample in tqdm.tqdm(demo_data_0)]
print(len(cleaned_sequences_opt_coqa))

import pickle
with open('cleaned_sequences_opt_coqa.pkl', 'wb') as fp:
    pickle.dump(cleaned_sequences_opt_coqa, fp)

opt_coqa = pd.read_pickle('cleaned_sequences_opt_coqa.pkl')
print(len(opt_coqa), opt_coqa[0])

7983


100%|████████████████████████████████████| 7983/7983 [00:00<00:00, 13939.85it/s]


7983
7983 {'prompt': tensor([    2, 11475,  2115,    10,    86,     6,    11,    10, 15909,   583,
           10,  3380,   790,     6,    89,  3033,    10,   410,  1104, 35292,
         1440, 15416,     4, 15416,  3033,   239,    62,    11,    10,  2579,
         3279,   317,  1065,     5, 15909,   147,    70,     9,     5, 10305,
           18,  8087, 17931,     4,   125, 15416,   938,    75,  1937,    11,
           69,   410,   184,  1065,     5, 15909,     6, 14223,   117,     4,
          264,  1373,    69, 13460,  3267,    19,    69,  3795,  4783,     8,
          195,    97,  7502,     4,   404,     9,    69,  7502,    58, 11962,
            8, 33564,     6,   101, 15416,     4,   125,    79,    21,     5,
          129,  1104,    65,    11,     5,  6900,     4,    20,  1079,     9,
           69,  7502,    58,    70,  8978,    19,  2721,  1104, 23921, 26224,
          101, 15416,    18,  3795,  4783,     4,  8374,   430,   156, 15416,
         1341,  5074,     4,   264,   747, 

In [2]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

import pipeline.uq_bb as uq_bb
reload(uq_bb)


obj = uq_bb.UQ_summ(demo_data, clean=True, split='test', cal_size=1000, seed=1)
# obj = uq_bb.UQ_summ(demo_data[:20], clean=True)

num_gens = 20
summ_kwargs = {
    'u+ea': {'overall': True, 'use_conf': False},
    'u+ia': {'overall': False, 'use_conf': False},
    'c+ia': {'overall': False, 'use_conf': True},
}['c+ia']

summ_obj = obj.summ([
'generations|numsets', 'lexical_sim',
    
        'generations|spectral_eigv_clip|disagreement_w',
        'generations|eccentricity|disagreement_w',
        'generations|degree|disagreement_w',

        'generations|spectral_eigv_clip|agreement_w',
        'generations|eccentricity|agreement_w',
        'generations|degree|agreement_w',


        'generations|spectral_eigv_clip|jaccard',
        'generations|eccentricity|jaccard',
        'generations|degree|jaccard',
    
#         'semanticEntropy|unnorm', 'self_prob',
], 
    
    acc_name='generations|rougeL|acc',
    num_gens=num_gens, **summ_kwargs
)

sum(summ_obj.summ_individual('auroc', use_conf=True)) / num_gens


2024-01-04 15:32:10.155171: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-01-04 15:32:10.942030: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


INFO: Pandarallel will run on 16 workers.
INFO: Pandarallel will use Memory file system to transfer data between the main process and workers.


╭─────────────────────────────── Traceback (most recent call last) ────────────────────────────────╮
│ in <module>:17                                                                                   │
│                                                                                                  │
│   14 │   'c+ia': {'overall': False, 'use_conf': True},                                           │
│   15 }['c+ia']                                                                                   │
│   16                                                                                             │
│ ❱ 17 summ_obj = obj.summ([                                                                       │
│   18 'generations|numsets', 'lexical_sim',                                                       │
│   19 │   │                                                                                       │
│   20 │   │   'generations|spectral_eigv_clip|disagreement_w',                                    │
│                                                                                                  │
│ /data/user-data/sa25729/GPT/UQ-NLG/pipeline/uq_bb.py:526 in summ                                 │
│                                                                                                  │
│   523 │   │   │   uq_kwargs = {}                                                                 │
│   524 │   │   │   if len(self.key) > 2:                                                          │
│   525 │   │   │   │   assert self.key[2] == 'test'                                               │
│ ❱ 526 │   │   │   │   self2 = self.__class__(self.path, self.key[1], 'val', self.key[3], self.   │
│   527 │   │   │   │   self2.tunable_hyperparams = {k:v for k, v in self.tunable_hyperparams.it   │
│   528 │   │   │   │   tuned_hyperparams = self2._tune_params(num_gens=num_gens,                  │
│   529 │   │   │   │   │   │   │   │   │   │   │   │   │   │    metric=acc_name,                  │
│                                                                                                  │
│ /data/user-data/sa25729/GPT/UQ-NLG/pipeline/uq_bb.py:406 in __init__                             │
│                                                                                                  │
│   403 │   def __init__(self, path, clean=False,                                                  │
│   404 │   │   │   │    split=None, cal_size=None, seed=None,                                     │
│   405 │   │   │   │    gpteval_examples = None) -> None:                                         │
│ ❱ 406 │   │   super().__init__(path, clean, split, cal_size, seed)                               │
│   407 │   │   self.gpteval_examples = gpteval_examples                                           │
│   408 │                                                                                          │
│   409 │   @functools.cached_property                                                             │
│                                                                                                  │
│ /data/user-data/sa25729/GPT/UQ-NLG/pipeline/uq_bb.py:153 in __init__                             │
│                                                                                                  │
│   150 │   │   │   self.key = (_clean_path(path), clean)                                          │
│   151 │   │   │   self.generations = dload.read_cleaned_outputs_new(path)                        │
│   152 │   │   else:                                                                              │
│ ❱ 153 │   │   │   assert isinstance(path, list)                                                  │
│   154 │   │   │   self.generations, self.path = path, None                                       │
│   155 │   │   │   self.key = (None, clean)                                                       │
│   156                                                      

In [ ]:
%cd UQ-NLG/
import sys; sys.path.append("..")
from importlib import reload
import persist_to_disk as ptd
import os
ptd.config.set_project_path(os.path.abspath("../"))
import tqdm
import pandas as pd
import numpy as np
import torch.nn as nn

import re
import torch

import utils

import matplotlib.pyplot as plt
%matplotlib inline

CONTRADICT, NEUTRAL, AGREE = 0, 1, 2
# logits: [Contradiction, neutral, entailment]
data_path = 'notebook/demo_data.pkl'

demo_data = pd.read_pickle(data_path)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

import pipeline.uq_bb as uq_bb
reload(uq_bb)


# obj = uq_bb.UQ_summ(path, clean=True, split='test', cal_size=1000, seed=1)
obj = uq_bb.UQ_summ(demo_data, clean=True)

num_gens = 20
summ_kwargs = {
    'u+ea': {'overall': True, 'use_conf': False},
    'u+ia': {'overall': False, 'use_conf': False},
    'c+ia': {'overall': False, 'use_conf': True},
}['c+ia']

summ_obj = obj.summ([
'generations|numsets', 'lexical_sim',
    
        'generations|spectral_eigv_clip|disagreement_w',
        'generations|eccentricity|disagreement_w',
        'generations|degree|disagreement_w',

        'generations|spectral_eigv_clip|agreement_w',
        'generations|eccentricity|agreement_w',
        'generations|degree|agreement_w',


        'generations|spectral_eigv_clip|jaccard',
        'generations|eccentricity|jaccard',
        'generations|degree|jaccard',
    
#         'semanticEntropy|unnorm', 'self_prob',
], 
    
    acc_name='generations|rougeL|acc',
    num_gens=num_gens, **summ_kwargs
)

sum(summ_obj.summ_individual('auroc', use_conf=True)) / num_gens


In [37]:
!nvidia-smi

Sat Jan  6 20:23:18 2024       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 525.147.05   Driver Version: 525.147.05   CUDA Version: 12.0     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla P40           Off  | 00000000:3B:00.0 Off |                    0 |
| N/A   32C    P0    49W / 250W |   1006MiB / 23040MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
|   1  Tesla P40           Off  | 00000000:AF:00.0 Off |                    0 |
| N/A   